Import Sentinel-1 Data from Google Earth Engine

This will download S1 data directly into your Google Drive.

Follow these steps for installing the EE library and troubleshooting authentication issues:
https://developers.google.com/earth-engine/guides/python_install

In [1]:
# import libraries

import ee
import geemap
from datetime import datetime, timedelta
import math
import matplotlib.pyplot as plt
import numpy as np
import ipynb
import time
from speckle_filter import leesigma

In [2]:
# get authentication credentials to initialize ee

ee.Authenticate()
ee.Initialize()

Enter verification code: 4/1AbUR2VODvwgVcQRcxhFA4A2oLmUCq7QeOnXq-f8PGfOoMEg4NLyfzk4fqUw

Successfully saved authorization token.


In [107]:
# define geometry and dates for data collection

# region of interest
roi = ee.Geometry.Rectangle([70.7980547394194417, 23.2880385161501735, 71.5060515087354958, 23.9024834537972986])  # Nanda Bet, India
#roi = ee.Geometry.Rectangle([-81.0, 25.4, -80.6, 25.6]) # Everglades

# dates of interest
d1 = '2020-08-03'
d2 = '2020-08-21'

In [108]:
# import s1 data

s1_collection = (ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT')
                 .filter(ee.Filter.eq('instrumentMode', 'IW'))
                 .filterMetadata('resolution_meters', 'equals', 10)
                 .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))
                 .filter(ee.Filter.date(d1,d2))
                 .filterBounds(roi))
print('Number of images in s1 collection:', s1_collection.size().getInfo())

Number of images in s1 collection: 2


In [109]:
# get dates of s1 images

acq_times = s1_collection.aggregate_array('system:time_start').getInfo()
date_list = [time.strftime('%m-%d-%Y', time.gmtime(t/1000)) for t in acq_times]
print(date_list)

['08-06-2020', '08-18-2020']


In [110]:
# implement speckle filter and get vv/vh ratio

def speckle(img):
    from speckle_filter import leesigma
    k = 5
    vv = img.select('VV')
    vh = img.select('VH')
    vv_vh = vv.divide(vh).rename('VV_VH')
    vv_filtered = leesigma(vv,k).rename('VV_filtered')
    vh_filtered = leesigma(vh,k).rename('VH_filtered')
    vv_vh_filtered = vv_filtered.divide(vh_filtered).rename('VV_VH_filtered')
    img = img.addBands(ee.Image([vv_vh, vv_filtered, vh_filtered, vv_vh_filtered]))
    return img

s1_collection = s1_collection.map(speckle)

In [111]:
# print band names

print(s1_collection.first().bandNames().getInfo())

['VV', 'VH', 'angle', 'VV_VH', 'VV_filtered', 'VH_filtered', 'VV_VH_filtered']


In [112]:
# export all n images in image collection

n = s1_collection.size().getInfo()  #number of images in image collection
s1_collection_list = s1_collection.toList(n)
scale = 100        #resolution of exported image in meters
maxPixels = 2e10  #needs to be high for high resolution or export will fail
task_id = []      #for tracking tasks

for i in range(0,n):
    img = ee.Image(s1_collection_list.get(i)).select(['VV_filtered','VH_filtered','VV_VH_filtered'])
    txt = 's1_' + date_list[i] + '_' + str(scale) + 'm'
    print(txt, date_list[i])
    task = ee.batch.Export.image.toDrive(image=img,
                                        description=txt,
                                        scale=scale,
                                        fileNamePrefix=txt,
                                        crs='EPSG:4326',
                                        fileFormat='GeoTIFF',
                                        maxPixels=maxPixels)
    task.start()
    task_id.append(task.id)
    

s1_08-06-2020_100m 08-06-2020
s1_08-18-2020_100m 08-18-2020


In [120]:
# list status of recent tasks

task.list()

[<Task LTU3NR6YAHYJISP3BZYNB5VZ EXPORT_IMAGE: s1_08-18-2020_100m (COMPLETED)>,
 <Task AQXDPEAVUHWAEEKMJAS5MQ4R EXPORT_IMAGE: s1_08-06-2020_100m (COMPLETED)>,
 <Task NE5ONN3Y2VTYI7QHF4XS4WSQ EXPORT_IMAGE: s1_08-06-2020_100m (COMPLETED)>,
 <Task 77725KKOSJ3UVABFF6BZBLTD EXPORT_IMAGE: s1_08-06-2020_100m (COMPLETED)>,
 <Task CEHWPR344YJETDR5JEKIASQJ EXPORT_IMAGE: s1_08-06-2020_100m (COMPLETED)>,
 <Task SUFLPX7AZTCOBPS3VW3AFXRW EXPORT_IMAGE: s1_08-30-2020_100m (COMPLETED)>,
 <Task KOTGCG2HZXEMIR3QMSESPBPX EXPORT_IMAGE: s1_08-18-2020_100m (COMPLETED)>,
 <Task R6OI7XOBQQWW4J6EF3TQEXYT EXPORT_IMAGE: s1_03-03-2020_100m (COMPLETED)>,
 <Task 2NPP2OAC3DYS2UI2UGA5UBHE EXPORT_IMAGE: s1_03-15-2020_10m (CANCELLED)>,
 <Task NDYV3I6JPSSZO4F66N3QN6RX EXPORT_IMAGE: s1_04-15-2020_10m (COMPLETED)>,
 <Task NNMA3F34BC6YCAPAZ7U7ZQW2 EXPORT_IMAGE: modis_ndvi_mean_LACounty_20220701_20220731 (COMPLETED)>]

In [55]:
task.cancel()

In [101]:
# map to make sure it looks right

# define visualization parameters
vv_vis = {
    'min': 0,
    'max': 0.2,
    'palette': ['#000000','#ffffff']
}

vh_vis = {
    'min': 0,
    'max': 0.03,
    'palette': ['#000000','#ffffff']
}

vv_vh_vis = {
    'min': 3,
    'max': 10,
    'palette': ['#000000','#ffffff']
}



# Select individual images to display on map
n = s1_collection.size().getInfo()
s1_collection_list = s1_collection.toList(n)

idx = 0  #get first image in collection
vv = ee.Image(s1_collection_list.get(idx)).select('VV_filtered')
vh = ee.Image(s1_collection_list.get(idx)).select('VH_filtered')
vv_vh = ee.Image(s1_collection_list.get(idx)).select('VV_VH_filtered')
vv_unfiltered = ee.Image(s1_collection_list.get(idx)).select('VV')


# Create map and add layers
M = geemap.Map()
M.centerObject(roi, 8)

M.addLayer(vv, vv_vis, 'VV')
M.addLayer(vh, vh_vis, 'VH')
M.addLayer(vv_vh, vv_vh_vis, 'VV/VH')
M.addLayer(vv_unfiltered, vv_vis, 'VV Unfiltered')

M  # show map


Map(center=[23.595422252032954, 71.15205312407748], controls=(WidgetControl(options=['position', 'transparent_…